# Inter-annotator agreement analyses


**To do:**

* Compare agreement between pairs with contradictory annotations and pairs without (of cleaned and uncleaned data)
* Remove workers with contradictions and check fails
* Test what happens to agreement if we collapse all annotations in a single group. 

**Done:**
* Remove pair-annotations if they are contradictory [done]
* Remove workers who fail checks [done]
* Remove workers with high contradictions [done]




## Relation evaluation:

* relations with most contradictions are probably difficult

## Example evaluation:

* Some examples may be missleading

In [1]:
# remove singletons

from load_data import load_experiment_data
from utils_analysis import sort_by_key

run = 3
batch = 16
n_q = 70
group = 'experiment1'

data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)

Discarded 10.0 annotations.


In [2]:

def remove_singletons(data_dict_list):
    clean_data = []
    data_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    for pair, dl in data_by_pair.items():
        data_by_relation = sort_by_key(dl, ['relation'])
        if len(data_by_relation) == 1:
            relation = list(data_by_relation.keys())[0]
            if relation != '_check' and not relation.startswith('test_'):
                continue
            else:
                clean_data.extend(dl)
        else:
            clean_data.extend(dl)
    return clean_data
        
clean_data = remove_singletons(data_dict_list)

In [18]:
print(len(clean_data))

780
